In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import math
import time

import numpy as np
from astropy import units as AstropyUnit
from astropy.time import Time as AstropyTime
from astropy.constants import Constant as AstropyConstant
from poliastro.bodies import Body as PoliastroBody
from poliastro.twobody import Orbit as PoliastroOrbit

import krpc
from krpc.client import Client
from typing import Union, NewType
Vessel = NewType("Vessel", object)
Body = NewType("Body", object)

from functools import reduce

conn = krpc.connect(name='jupyter')

def _convert_body_krpc_to_poliastro(poliastro_bodies: dict, parent: PoliastroBody, krpc_body: Body):
    name = krpc_body.name
    GM = AstropyConstant('GM_k{}'.format(name), 'Kerbal {} gravitational constant'.format(name),
                  krpc_body.gravitational_parameter, 'm3 / (s2)', 0,
                  'kRPC space_center.bodies["{}"].gravitational_parameter'.format(name), system='si')
    R = AstropyConstant('R_k{}'.format(name), 'Kerbal {} equatorial radius'.format(name),
                      krpc_body.equatorial_radius, 'm', 0,
                     'kRPC space_center.bodies["{}"].equatorial_radius'.format(name), system='si')
    poliastro_body = PoliastroBody(parent, GM, "", name, R)
    poliastro_bodies[name] = poliastro_body
    for satelite in krpc_body.satellites:
        _convert_body_krpc_to_poliastro(poliastro_bodies, poliastro_body, satelite)
    return

def krpc_poliastro_bodies(conn: Client) -> (dict, dict):
    poliastro_bodies = {}
    krpc_bodies = conn.space_center.bodies

    krpc_Sun = krpc_bodies["Sun"]
    _convert_body_krpc_to_poliastro(poliastro_bodies, None, krpc_Sun)
    return (krpc_bodies, poliastro_bodies)


krpc_bodies, poliastro_bodies = krpc_poliastro_bodies(conn)

In [2]:
krpc_ut = conn.space_center.ut
astropy_ut = AstropyTime(krpc_ut, format="unix")

vessel = conn.space_center.active_vessel
attractor = vessel.orbit.body
if conn.space_center.target_body:
    target = conn.space_center.target_body
    target_type = "CelestialBody"
elif conn.space_center.target_vessel:
    target = conn.space_center.target_vessel
    target_type = "Vessel"

reference_frame = krpc_bodies[attractor.name].non_rotating_reference_frame

r_target = target.position(reference_frame) * AstropyUnit.m
v_target = target.velocity(reference_frame) * AstropyUnit.m / AstropyUnit.s
ss_target = PoliastroOrbit.from_vectors(poliastro_bodies[attractor.name], r_target, v_target, epoch=astropy_ut)

r_i = vessel.position(reference_frame) * AstropyUnit.m
v_i = vessel.velocity(reference_frame) * AstropyUnit.m / AstropyUnit.s
ss_i = PoliastroOrbit.from_vectors(poliastro_bodies[attractor.name], r_i, v_i, epoch=astropy_ut)

NameError: name 'target' is not defined

In [ ]:
from poliastro.plotting import plot
plot(ss_target)

In [ ]:
from poliastro.maneuver import Maneuver
ap_target = target.orbit.apoapsis
hoh = Maneuver.hohmann(ss_i, ap_target * AstropyUnit.m)
hoh.get_total_cost()

In [ ]:
np.linalg.norm(hoh[0][1])

In [ ]:
hoh.get_total_time().value

In [ ]:
from poliastro.plotting import OrbitPlotter

op = OrbitPlotter()
ss_a, ss_f = ss_i.apply_maneuver(hoh, intermediate=True)
op.plot(ss_i, label="Initial orbit")
op.plot(ss_a, label="Transfer orbit")
op.plot(ss_f, label="Final orbit")

In [ ]:
from poliastro.util import norm
delta_v = norm(tuple(val.decompose([AstropyUnit.m, AstropyUnit.s]) for val in hoh[0])[1])

In [ ]:
ss_i.sample(1)

In [ ]:
ss_a.propagate(hoh.get_total_time()).sample(1)

In [ ]:
ss_target.propagate(hoh.get_total_time()).sample(1)

In [ ]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def dot(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [ ]:
angle_between(ss_a.propagate(hoh.get_total_time()).sample(1).xyz.value.take([0,1,2]),
              ss_target.propagate(hoh.get_total_time()).sample(1).xyz.value.take([0,1,2]))

In [ ]:
def hohmann_transfer_to_target_at_ut(vessel: Vessel, target: Union[Vessel, Body], ct: float, ut: float, trans_time: float = 0) -> ((float, float), float, float):
    attractor = vessel.orbit.body
    reference_frame = attractor.non_rotating_reference_frame

    krpc_ut = ut
    krpc_ct = conn.space_center.ut
    astropy_ct = AstropyTime(krpc_ct, format="unix")
    krpc_time_to_ut = krpc_ut - krpc_ct
    astropy_time_to_ut = krpc_time_to_ut * AstropyUnit.s
    
    r_target = target.position(reference_frame) * AstropyUnit.m
    v_target = target.velocity(reference_frame) * AstropyUnit.m / AstropyUnit.s
    ss_target = PoliastroOrbit.from_vectors(poliastro_bodies[attractor.name], r_target, v_target, epoch=astropy_ct)

    r_v_ct = vessel.position(reference_frame) * AstropyUnit.m
    v_v_ct = vessel.velocity(reference_frame) * AstropyUnit.m / AstropyUnit.s
    ss_v_ct = PoliastroOrbit.from_vectors(poliastro_bodies[attractor.name], r_i, v_i, epoch=astropy_ct)

    ss_i = ss_v_ct.propagate(astropy_time_to_ut)
    r_f = target.orbit.radius_at(krpc_ut + trans_time) * AstropyUnit.m

    hoh = Maneuver.hohmann(ss_i, r_f)
    
    trans_time = hoh.get_total_time().value
    ss_a, ss_f = ss_i.apply_maneuver(hoh, intermediate=True)
    
    dv_a = np.linalg.norm(hoh[0][1])
    dv_b = np.linalg.norm(hoh[1][1])
    
    phase_angle = angle_between(
        ss_a.propagate(hoh.get_total_time()).sample(1).xyz.value.take([0,1,2]),
        ss_target.propagate(hoh.get_total_time()).sample(1).xyz.value.take([0,1,2])
    )
    
    return (dv_a, dv_b, trans_time, phase_angle)

In [ ]:
1 * AstropyUnit.s

In [5]:
a = np.subtract([0,1,2], [2,3,4])
a

array([-2, -2, -2])

In [6]:
a[0]

-2